In [1]:
import requests

In [2]:
import geopandas as gpd
import pandas as pd

## Open State data

In [3]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Household Data

In [10]:
url = "https://api.census.gov/data/2023/acs/acs5/subject"
params = params = {
    "get": "NAME,S1101_C01_002E",
    "for": "state:*"
}
response = requests.get(url, params=params)

In [12]:
data = response.json()
columns = data[0]
rows = data[1:]
household_df = pd.DataFrame(rows, columns=columns)

In [21]:
household_df = household_df.rename(columns={"state": "STATEFP", "S1101_C01_002E":"HOUSEHOLD_SIZE"})

## Merge Data

In [22]:
household_gdf = states_df.merge(household_df, how='inner', on='STATEFP')

In [25]:
household_gdf = household_gdf[['geometry', 'HOUSEHOLD_SIZE']]
household_gdf['HOUSEHOLD_SIZE'] = household_gdf['HOUSEHOLD_SIZE'].astype(float)

In [26]:
household_gdf = household_gdf.to_crs(9311)
household_gdf.to_file("data/Household_size_per_State.gpkg")